In [345]:
import numpy as np
import os
from sklearn import tree
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
#from sklearn.model_selection import cross_validate

folderPath = '../stage1'
trainArray = np.load(os.path.join(folderPath,'train.npy'))
testArray = np.load(os.path.join(folderPath, 'test.npy'))

trainData = trainArray[:,0:-1]
trainLabel = trainArray[:,-1]

testData = testArray[:,0:-1]
testLabel = testArray[:,-1]

# trainData = trainData[:,0:10]
# testData = testData[:,0:10]

# trainData = np.delete(trainData, 9, axis=1)
# testData = np.delete(testData, 9, axis=1)

## train split for debugging
testMetaDataFile = '../stage1/trainMetaData.txt'
testMetadata = []
for line in open(testMetaDataFile).readlines():
    testMetadata.append(line)

import random
#80-20 split
nValSize = int(0.2*trainData.shape[0])
nTrainSize =trainData.shape[0] - nValSize
splitIdxs = random.sample(range(0,trainData.shape[0]), nValSize)

valMetaMapping = { i : sidx for i,sidx in enumerate(splitIdxs) }

nFeatures = 62
nValData = np.zeros((nValSize,nFeatures))
nValLabel = np.zeros((nValSize))

nTrainData = np.zeros((nTrainSize,nFeatures))
nTrainLabel = np.zeros((nTrainSize))

tId = 0
valId = 0
valMetaData = []
for idx in range(trainData.shape[0]):
    if idx in splitIdxs:
        nValData[valId,:] = trainData[idx,:]
        nValLabel[valId] = trainLabel[idx]
        valId += 1
        valMetaData.append(testMetadata[idx])
    else:
        nTrainData[tId, :] = trainData[idx,:]
        nTrainLabel[tId] = trainLabel[idx]
        tId += 1

#print (valMetaMapping)

In [353]:
testMetaDataFile = '../stage1/testMetaData.txt'
testMetaData = []
for line in open(testMetaDataFile).readlines():
    testMetaData.append(line)

print (len(testMetaData))

14791


In [313]:
dt = tree.DecisionTreeClassifier(min_samples_split=10, random_state=99)
scores = cross_val_score(dt, trainData, trainLabel, scoring='precision', cv=5)
#scoring = ['precision_macro', 'recall_macro']
#scores = cross_validate(dt, trainData, trainLabel, scoring=scoring, cv=5, return_train_score=False)
print (scores)
dt = dt.fit(trainData, trainLabel)
pred = dt.predict(testData)
prec = precision_score(testLabel, pred)
recall = recall_score(testLabel, pred)
f1Score = f1_score(testLabel, pred)
print (prec)
print (recall)
print (f1Score)

# from IPython.display import Image  
# import pydotplus
# from sklearn.externals.six import StringIO
# dot_data = StringIO()
# tree.export_graphviz(dt, out_file=dot_data,  filled=True, rounded=True, special_characters=True)
# graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
# Image(graph.create_png())

[ 0.71280992  0.67039106  0.7231405   0.68224299  0.69047619]
0.685641461342
0.663105998357
0.674185463659


In [410]:
from sklearn.ensemble import RandomForestClassifier
RF = RandomForestClassifier(n_estimators=150, max_depth=50,  random_state=99)
# scores = cross_val_score(RF, trainData, trainLabel, scoring='precision', cv=5)
# print (scores)

RF = RF.fit(trainData, trainLabel)
pred = RF.predict(testData)

# for i in range(0,len(testMetaData)):
#     token = (testMetaData[i].split(','))
#     #print (token[3] + token[3][-1])

#     if ((token[3][-3:-1] == "ed") or (token[4][-3:-1] == 'ed')) and (testData[i,0] == 1) and (testData[i,5] == 2) and (testData[i,1] == 0) and (testData[i,2] == 0):
#         if int(token[6].strip(']\n')) == 0:
#             print (token)
#         pred[i] = 1


# prec = precision_score(testLabel, pred)
# recall = recall_score(testLabel, pred)
# f1Score = f1_score(testLabel, pred)

RF = RF.fit(nTrainData, nTrainLabel)
pred = RF.predict(nValData)
prec = precision_score(nValLabel, pred)
recall = recall_score(nValLabel, pred)
f1Score = f1_score(nValLabel, pred)

print (prec)
print (recall)
print (f1Score)

0.941489361702
0.651933701657
0.770402611534


In [413]:
countPos = 0
countNeg = 0
countTP = 0
countFN = 0
countFP = 0
index = 0;
wrongList = []
for p,t in zip(pred,nValLabel):
#for p,t in zip(pred,testLabel):
    if p!=t:
        countNeg+=1
    else :
        countPos+=1
    if t==1 and p==t:
        countTP+=1
    if t==1 and p!=t:
        countFN+=1
        wrongList.append(index)
#         token = (testMetaData[index].split(','))
#         if ((token[3][-3:-1] == "ed") or (token[4][-3:-1] == 'ed')) and (testData[index,0] == 1):
    if t==0 and p!=t:
        countFP+=1
        wrongList.append(index)
    index+=1
print(countPos,countNeg,float(countPos)/(countPos+countNeg))
print("True Pos:",countTP)
print("Fals Pos:",countFP)
print("Fals Neg:",countFN)
for i in wrongList:
    string = str(valMetaData[i])
    print(string,end='')

5992 211 0.9659842011929711
True Pos: 354
Fals Pos: 22
Fals Neg: 189
[['[[Rusedski]] Serve', 89, 90], 'over', 'was', '../stage1/train\\130.txt', 0]
[['[[Duff]]', 237, 237], 'Forest', 'Chelsea', '../stage1/train\\102.txt', 1]
[['Brive', 298, 298], 'while', 'back', '../stage1/train\\098.txt', 0]
[["[[Eddie O'Sullivan]]", 98, 99], 'coach', 'In', '../stage1/train\\071.txt', 1]
[['[[Ceplak]]', 195, 195], 'Meanwhile', 'will', '../stage1/train\\085.txt', 1]
[['[[Victor]]', 170, 170], 'about', 'is', '../stage1/train\\203.txt', 1]
[['[[J Dunbar]]', 274, 275], 'Borders', 'Leeds', '../stage1/train\\298.txt', 1]
[['[[Campbell]]', 250, 250], 'Ramsay]]', 'is', '../stage1/train\\154.txt', 1]
[['[[Edwards]]', 168, 168], 'Sunderland', 'Wolves', '../stage1/train\\051.txt', 1]
[['[[Horgan]]', 155, 155], '[[Murphy]]', 'TBC', '../stage1/train\\151.txt', 1]
[['Minister', 0, 0], '', 'digs', '../stage1/train\\159.txt', 0]
[['[[Pierre Mignoni]]', 229, 230], 'Yachvili]]', '[[Frederic', '../stage1/train\\138.txt

In [118]:
from sklearn import svm
bSVM = svm.SVC(class_weight='balanced')
#bSVM = svm.LinearSVC()

#scores = cross_val_score(bSVM, trainData, trainLabel, scoring='precision', cv=5)
#print (scores)
bSVM = bSVM.fit(trainData, trainLabel)
pred = bSVM.predict(testData)
prec = precision_score(testLabel, pred)
recall = recall_score(testLabel, pred)
f1Score = f1_score(testLabel, pred)
print (prec)
print (recall)
print (f1Score)

'''
rbf with balanced
0.96169837914
0.454844006568
0.504783599089'''

0.5670419652
0.454844006568
0.504783599089


'\nrbf with balanced\n0.96169837914\n0.454844006568\n0.504783599089'

In [124]:
from sklearn.linear_model import LinearRegression
LR = LinearRegression()

#scores = cross_val_score(LR, trainData, trainLabel, scoring='precision', cv=5)
#print (scores)

LR = LR.fit(trainData, trainLabel)
pred = LR.predict(testData)
pred = np.where(pred > 0.5, 1, 0)
prec = precision_score(testLabel, pred)
recall = recall_score(testLabel, pred)
f1Score = f1_score(testLabel, pred)
print (prec)
print (recall)
print (f1Score)

0.576763485477
0.114121510673
0.190541466758


In [125]:
from sklearn.linear_model import LogisticRegression
LogR = LogisticRegression(random_state=99, class_weight='balanced', penalty='l1', C=10000)

scores = cross_val_score(LogR, trainData, trainLabel, scoring='precision', cv=5)
print (scores)

LogR = LogR.fit(trainData, trainLabel)
predProb = LogR.predict_proba(testData)

pred = LogR.predict(testData)
#pred = np.zeros(testData.shape[0])
#pred = np.where(predProb[:,1] > 0.5, 1, 0)

#pred = np.where(pred >= 0.5, 1, 0)
prec = precision_score(testLabel, pred)
recall = recall_score(testLabel, pred)
f1Score = f1_score(testLabel, pred,average='macro')
print (prec)
print (recall)
print (f1Score)

[ 0.35584065  0.35989196  0.36032389  0.34826203  0.36271186]
0.215476621964
0.97619047619
0.632955160145


In [64]:
LogR.classes_

array([ 0.,  1.])

In [65]:
countPos = 0
countNeg = 0
countTP = 0
countFN = 0
countFP = 0
index = 0;
wrongList = []
for p,t in zip(pred,testLabel):
    if p!=t:
        countNeg+=1
    else :
        countPos+=1
    if t==1 and p==t:
        countTP+=1
    if t==1 and p!=t:
        countFN+=1
#         wrongList.append(index)
    if t==0 and p!=t:
        countFP+=1
        wrongList.append(index)
    index+=1
print(countPos,countNeg,float(countPos)/(countPos+countNeg))
print("True Pos:",countTP)
print("Fals Pos:",countFP)
print("Fals Neg:",countFN)
for i in wrongList:
    print(testMetadata[i],end='')

25749 2631 0.9072938689217759
True Pos: 1029
Fals Pos: 2442
Fals Neg: 189
[['Real', 3, 3], 'with', 'display', '../stage1/test\\297.txt']
[['Euro', 4, 4], 'retain', 'lure', '../stage1/test\\211.txt']
[['Nations', 121, 121], 'of', 'qualifier', '../stage1/test\\167.txt']
[['Sri Lanka', 0, 1], '', 'squad', '../stage1/test\\068.txt']
[['World Cup', 106, 107], 'the', 'final', '../stage1/test\\188.txt']
[['Wales', 186, 186], 'against', 'and', '../stage1/test\\134.txt']
[['Wimbledon', 72, 72], 'the', 'final', '../stage1/test\\100.txt']
[['British', 147, 147], 'of', 'athletes', '../stage1/test\\106.txt']
[['England', 253, 253], 'with', 'and', '../stage1/test\\188.txt']
[['BBC', 233, 233], 'told', 'Radio', '../stage1/test\\015.txt']
[['Milan', 100, 100], 'in', 'talking', '../stage1/test\\299.txt']
[['Bowlers', 31, 31], 'cricket', 'will', '../stage1/test\\039.txt']
[['Park', 291, 291], 'Celtic', '', '../stage1/test\\252.txt']
[['Japan [[Brian Kerr]]', 138, 140], 'Sapporo', 'said', '../stage1/test

[['Wilkinson]] [[Mike Tindall]]', 120, 122], '[[Jonny', '[[Stuart', '../stage1/test\\212.txt']
[['[[M', 145, 145], 'capt', 'Cueto]]', '../stage1/test\\212.txt']
[['[[Davis]] Cup', 64, 65], 'the', 'team', '../stage1/test\\121.txt']
[['Melbourne Cricket', 222, 223], 'into', 'Ground', '../stage1/test\\148.txt']
[['BBC', 18, 18], 'Ferguson]]', 'Sport', '../stage1/test\\114.txt']
[['Swiss', 74, 74], 'the', 'star', '../stage1/test\\164.txt']
[['Charlton [[Arnesen]]', 126, 127], 'against', 'said', '../stage1/test\\058.txt']
[['Belgian', 18, 18], 'The', 'has', '../stage1/test\\055.txt']
[['[[Rio', 46, 46], 'Injured', 'Ferdinand]]', '../stage1/test\\179.txt']
[['Both', 29, 29], 'game', 'were', '../stage1/test\\062.txt']
[['[[Richie', 149, 149], 'skipper', 'Pugh]]', '../stage1/test\\001.txt']
[['Group Six', 95, 96], 'in', 'with', '../stage1/test\\150.txt']
[['Twickenham', 40, 40], 'at', 'on', '../stage1/test\\134.txt']
[['Australia', 11, 11], 'tour', 'after', '../stage1/test\\169.txt']
[['Their'

In [12]:
'''
countPos = 0
countNeg = 0
countTP = 0
index = 0;
for p,t in zip(pred,testLabel):
    if p!=t:
        countNeg+=1
    else :
        countPos+=1
    if t==1 and p==t:
        countTP+=1
    if t==1 and p!=t:
        print(index)
    index+=1
print(countPos,countNeg,float(countPos)/(countPos+countNeg))
print(countTP)
'''

'\ncountPos = 0\ncountNeg = 0\ncountTP = 0\nindex = 0;\nfor p,t in zip(pred,testLabel):\n    if p!=t:\n        countNeg+=1\n    else :\n        countPos+=1\n    if t==1 and p==t:\n        countTP+=1\n    if t==1 and p!=t:\n        print(index)\n    index+=1\nprint(countPos,countNeg,float(countPos)/(countPos+countNeg))\nprint(countTP)\n'